Welcome to exercise three of “Apache Spark for Scalable Machine Learning on BigData”. In this exercise you’ll create a DataFrame, register a temporary query table and issue SQL commands against it. 

Let’s create a little data frame:

%%writefile my_csv_file.csv
Stat.Suffix,Article Description, Unit of Quantity ,ID
,Travel clocks,      9105.29.30
10,Movement    No.,9105.99.40
20,Case,   No., 9105.11.10
    Other clocks:,       
30,Movement    No. 
40,Case    No. 

,Stat.Suffix,Article Description,Unit of Quantity,ID
0,NaN,Travel clocks,NaN,9105.29.30
1,10.0,Movement,No.,9105.99.40
2,20.0,Case,No.,9105.11.10
3,NaN,Other clocks:,NaN,NaN
4,30.0,Movement,No.,NaN
5,40.0,Case,No.,NaN


In [36]:
import pandas

my_data_frame = pandas.read_csv('/Users/jordanwillis/Book1.csv')

#get all ids
ids = list(my_data_frame[~my_data_frame['ID'].isna()]['ID'])

#have your dataframe with ID removed
my_data_frame = my_data_frame.drop('ID',axis=1)

merged_df = []
for i in ids:
    #copy your dataframe
    tmp_df = my_data_frame.copy()
    stats_df = tmp_df.loc[~tmp_df['Stat.Suffix'].isna()]
    
    stats_df['Stat.Suffix'] = .apply(lambda x: str(''))
    for index,row in my_data_frame.loc[~my_data_frame['Stat.Suffix'].isna()].iterrows():
        tmp_df.loc[index,'Stat.Suffix'] = str(row['Stat.Suffix']) +'.' + str(i)
    merged_df.append(tmp_df)
my_df = pandas.concat(merged_df[::-1]).reset_index(drop=True)
my_df

,Stat.Suffix,Article Description,Unit of Quantity
0,NaN,Travel clocks,NaN
1,10.0.9105.11.10,Movement,No.
2,20.0.9105.11.10,Case,No.
3,NaN,Other clocks:,NaN
4,30.0.9105.11.10,Movement,No.
5,40.0.9105.11.10,Case,No.
6,NaN,Travel clocks,NaN
7,10.0.9105.99.40,Movement,No.
8,20.0.9105.99.40,Case,No.
9,NaN,Other clocks:,NaN


,Stat.Suffix,Article Description,Unit of Quantity
0,NaN,Travel clocks,NaN
1,10.0.9105.11.10,Movement,No.
2,20.0.9105.11.10,Case,No.
3,NaN,Other clocks:,NaN
4,30.0.9105.11.10,Movement,No.
5,40.0.9105.11.10,Case,No.
0,NaN,Travel clocks,NaN
1,10.0.9105.99.40,Movement,No.
2,20.0.9105.99.40,Case,No.
3,NaN,Other clocks:,NaN


In [13]:
my_data_frame.loc[~my_data_frame['Stat.Suffix'].isna()]

,Stat.Suffix,Article Description,Unit of Quantity,"(1, Stat.Suffix)","(2, Stat.Suffix)","(4, Stat.Suffix)","(5, Stat.Suffix)"
1,10.0,Movement,No.,10.0.9105.11.10,20.0.9105.11.10,30.0.9105.11.10,40.0.9105.11.10
2,20.0,Case,No.,10.0.9105.11.10,20.0.9105.11.10,30.0.9105.11.10,40.0.9105.11.10
4,30.0,Movement,No.,10.0.9105.11.10,20.0.9105.11.10,30.0.9105.11.10,40.0.9105.11.10
5,40.0,Case,No.,10.0.9105.11.10,20.0.9105.11.10,30.0.9105.11.10,40.0.9105.11.10


In [1]:
from pyspark import SparkConf
from pyspark import SparkContext

conf = SparkConf()

sc = SparkContext(conf=conf)
from pyspark.sql import Row
from pyspark.sql import SparkSession

spark = SparkSession(sc)

    


In [ ]:
spark.createDataFrame

In [2]:
df = spark.createDataFrame([Row(id=1, value='value1'),Row(id=2, value='value2')])

# let's have a look what's inside
df.show()

# let's print the schema
df.printSchema()

+---+------+
| id| value|
+---+------+
|  1|value1|
|  2|value2|
+---+------+

root
 |-- id: long (nullable = true)
 |-- value: string (nullable = true)



Now we register this DataFrame as query table and issue an SQL statement against it. Please note that the result of the SQL execution returns a new DataFrame we can work with.

In [4]:
# register dataframe as query table
df.createOrReplaceTempView('df_view')

# execute SQL query
df_result = spark.sql('select value from df_view where id=2')

# examine contents of result
df_result.show()

# get result as string
df_result.first().value

+------+
| value|
+------+
|value2|
+------+



'value2'

Although we’ll learn more about DataFrames next week, please try to find a way to count the rows in this DataFrame by looking at the API documentation. No worries, we’ll cover DataFrames in more detail next week.

https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame

In [5]:
df.count()

2